In [5]:
from etoe_unet import *

In [6]:
image_db = etoe_unet.TrainingDB()
image_db.build_training_db(IMAGE_SF,  sorted(DF.ImageId.unique()), N_Cls, subset_size=0)
etoe_unet.build_and_display_thumbs_from_image_list(image_db.x)
etoe_unet.build_and_display_thumbs_from_image_list([img[:, :, 0:3] for img in image_db.y])
etoe_unet.build_and_display_thumbs_from_image_list([img[:, :, 3:6] for img in image_db.y])

1/25 6010_1_2 shapes: (2519, 2555, 3) (2519, 2555, 10)


2/25 6010_4_2 shapes: (2516, 2555, 3) (2516, 2555, 10)


3/25 6010_4_4 shapes: (2516, 2552, 3) (2516, 2552, 10)


4/25 6040_1_0 shapes: (2519, 2551, 3) (2519, 2551, 10)


5/25 6040_1_3 shapes: (2519, 2551, 3) (2519, 2551, 10)


...

23/25 6170_0_4 shapes: (2519, 2549, 3) (2519, 2549, 10)


24/25 6170_2_4 shapes: (2519, 2549, 3) (2519, 2549, 10)


25/25 6170_4_1 shapes: (2516, 2552, 3) (2516, 2552, 10)
2520 2560 5 (12600, 12800, 3)


2520 2560 5 (12600, 12800, 3)


2520 2560 5 (12600, 12800, 3)


In [7]:
model = get_unet()
model.load_weights(config.glb_base_dir + '/weights/model-17-0.02772.hdf5')

In [20]:
test_id_list = sorted(set(SB['ImageId'].tolist()))
image_id = test_id_list[0]
img_mat = image_source.get_image_from_id(image_id, IMAGE_SF)
img_mat = image_source.stretch_n(img_mat)
print('test image', img_mat.shape)
img_prob = map_image_to_probability_matrix(img_mat, model)
print('prob shape', img_prob.shape)

test image (2519, 2555, 3)


prob shape (10, 2519, 2555)


In [21]:
print(np.min(img_prob), np.max(img_prob))

2.71033e-30 0.999996


In [43]:
from pylab import rcParams
rcParams['figure.figsize'] = 11, 12

In [51]:
# Give short names to object types
CLASSES = {
        0: 'Bldg',
        1: 'Structure',
        2: 'Road',
        3: 'Track',
        4: 'Trees',
        5: 'Crops',
        6: 'Fast H20',
        7: 'Slow H20',
        8: 'Truck',
        9: 'Car',
        }

for i in range(10):
    plt.imshow(img_prob[i, :, :], cmap='gray')  # other cmap: hot, pink, cool, bone, afmhot
    plt.title(CLASSES[i])
    plt.show()

In [50]:
train_id_list = sorted(DF.ImageId.unique())
image_id = train_id_list[0]
img_mat = image_source.get_image_from_id(image_id, IMAGE_SF)
img_mat = image_source.stretch_n(img_mat)
print('test image', img_mat.shape)
img_prob = map_image_to_probability_matrix(img_mat, model)
print('prob shape', img_prob.shape)

test image (2519, 2555, 3)


prob shape (10, 2519, 2555)


Code for computing optimum thresholds for binarization using Jaccard.

In [72]:
train_index = 1  # 0 .. 24
y_true = image_db.y[train_index]
img_mat = image_db.x[train_index]
print(y_true.shape, img_mat.shape)
y_pred = map_image_to_probability_matrix(img_mat, model)
print(y_pred.shape)

(2516, 2555, 10) (2516, 2555, 3)


(10, 2516, 2555)


In [74]:
num_thresh = 20
best_jacc = np.zeros(N_Cls)
best_thresh = np.zeros(N_Cls)
for ic in range(N_Cls):
    for j in range(num_thresh):
        thresh = float(j) / num_thresh
        y_pred_thresh = y_pred[ic, :, :] > thresh
        jacc = jaccard_tools.jaccard_similarity_score(y_true[:, :, ic], y_pred_thresh)
        if jacc > best_jacc[ic]:
            best_jacc[ic] = jacc
            best_thresh[ic] = thresh
    print(j, ic, 'thresh', thresh, 'jacc', jacc, 'best', best_thresh[ic], best_jacc[ic])

19 0 thresh 0.95 jacc 0.85413354531 best 0.95 0.85413354531


19 1 thresh 0.95 jacc 0.99960254372 best 0.95 0.99960254372


19 2 thresh 0.95 jacc 0.860492845787 best 0.95 0.860492845787


19 3 thresh 0.95 jacc 0.0 best 0.1 0.0399537940091


19 4 thresh 0.95 jacc 0.289349741355 best 0.45 0.630629247531


19 5 thresh 0.95 jacc 0.0242448330684 best 0.95 0.0242448330684


19 6 thresh 0.95 jacc 0.990858505564 best 0.95 0.990858505564


19 7 thresh 0.95 jacc 0.91613672496 best 0.95 0.91613672496


19 8 thresh 0.95 jacc 1.0 best 0.3 1.0


19 9 thresh 0.95 jacc 1.0 best 0.75 1.0


In [75]:
best_jacc

array([ 0.85413355,  0.99960254,  0.86049285,  0.03995379,  0.63062925,
        0.02424483,  0.99085851,  0.91613672,  1.        ,  1.        ])

In [76]:
best_thresh

array([ 0.95,  0.95,  0.95,  0.1 ,  0.45,  0.95,  0.95,  0.95,  0.3 ,  0.75])

In [81]:
jacc_int = jaccard_tools.jaccard_coef_int(y_true, y_pred)
print('jacc_int', jacc_int)
for ic in range(N_Cls):
    y_pred_thresh = y_pred[ic, :, :] > best_thresh[ic]
    y_true_class = y_true[:, :, ic]
    jacc = jaccard_tools.jaccard_similarity_score(y_true_class, y_pred_thresh)
    my_jacc = np.sum(y_pred_thresh * y_true[:, :, ic])
    print('jacc', jacc, 'my_jacc', my_jacc, 'jacc_int', jacc_int, 'sum', np.sum(y_pred_thresh), 
          'of true sum', np.sum(y_true_class))

jacc_int mean


jacc 0.85413354531 my_jacc 0 jacc_int mean sum 1583 of true sum 0


jacc 0.99960254372 my_jacc 0 jacc_int mean sum 2 of true sum 0


jacc 0.860492845787 my_jacc 0 jacc_int mean sum 5111 of true sum 0


jacc 0.0399537940091 my_jacc 18592 jacc_int mean sum 268261 of true sum 135556


...

jacc 0.0242448330684 my_jacc 0 jacc_int mean sum 291547 of true sum 0


jacc 0.990858505564 my_jacc 0 jacc_int mean sum 145 of true sum 0


jacc 0.91613672496 my_jacc 0 jacc_int mean sum 1460 of true sum 0


jacc 1.0 my_jacc 0 jacc_int mean sum 0 of true sum 0


jacc 1.0 my_jacc 0 jacc_int mean sum 0 of true sum 0
